# 목표

- 관세청 영문 문장 데이터 추가

# Library

In [1]:
# visualization
import matplotlib
matplotlib.rcParams['axes.unicode_minus'] = False
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
fe = fm.FontEntry(
    fname=r'.venv/Lib/site-packages/matplotlib/mpl-data/fonts/ttf/NanumGothic.ttf', # ttf 파일이 저장되어 있는 경로
    name='NanumBarunGothic')                        # 이 폰트의 원하는 이름 설정
fm.fontManager.ttflist.insert(0, fe)              # Matplotlib에 폰트 추가
plt.rcParams.update({'font.size': 10, 'font.family': 'NanumBarunGothic'}) # 폰트 설정
plt.rc('font', family='NanumBarunGothic')
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import missingno as msno

In [2]:
import os
import re
import json
import yaml
import time
import numpy as np
import pandas as pd

from tqdm import tqdm
from pprint import pprint

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.common.by import By

# Data Load

In [3]:
text_data = pd.read_csv('./data/영문_텍스트.csv', dtype=str)
statistics_data = pd.read_csv('./data/통계청.csv', dtype=str)
customs_data = pd.read_csv('./data/관세청.csv', dtype=str)

In [4]:
text_data_copy = text_data.copy()
statistics_data_copy = statistics_data.copy()
customs_data_copy = customs_data.copy()

In [5]:
display(text_data_copy.info())
display(statistics_data_copy.info())
display(customs_data_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   ID      10000 non-null  object
 1   CODE    10000 non-null  object
 2   DSC     10000 non-null  object
dtypes: object(3)
memory usage: 234.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6848 entries, 0 to 6847
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   ISIC4_국제표준산업분류     2394 non-null   object
 1   ISIC4_분류명          2393 non-null   object
 2   KSIC10_한국표준산업분류    2933 non-null   object
 3   KSIC10_분류명         2930 non-null   object
 4   HS2017_관세통계통합품목분류  5637 non-null   object
 5   HS2017_분류명         5637 non-null   object
dtypes: object(6)
memory usage: 321.1+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12422 entries, 0 to 12421
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호       12422 non-null  object
 1   한글품목명      12422 non-null  object
 2   영문품목명      12422 non-null  object
 3   성질통합분류코드   11294 non-null  object
 4   성질통합분류코드명  11294 non-null  object
dtypes: object(5)
memory usage: 485.4+ KB


None

In [ ]:
customs_data_copy['HS부호'].apply(lambda x: x[:4])

In [ ]:
customs_data_copy['HS부호_앞4자리'] = customs_data_copy['HS부호'].apply(lambda x: x[:4])

In [ ]:
print(customs_data_copy.shape[0])
print(customs_data_copy['HS부호_앞4자리'].nunique())

# 앞 4자리로 기준 잡으면 됨 (앞4자리, 앞6자리 비교했는데 앞4자리가 공통되는 기준임)

In [ ]:
customs_data_copy['HS부호_앞4자리'].unique()

In [ ]:
customs_data_copy[customs_data_copy['HS부호_앞4자리'] == '0101']

- 0101, 010121, 0101211000의 설명은 같음

In [ ]:
# -------------------- 함수 모음 --------------------
## 스크롤 끝까지
def scroll(driver):
    scroll_location = driver.execute_script('return document.body.scrollHeight')
    while True:
        driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
        time.sleep(2)
        scroll_height = driver.execute_script('return document.body.scrollHeight')
        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script('return document.body.scrollHeight')
    driver.implicitly_wait(3)


## 스크롤 한 번만
def scroll_one(driver):
    driver.execute_script('window.scrollTo(0,document.body.scrollHeight)')
    time.sleep(1)
    driver.implicitly_wait(3)

## 스크롤 위로
def scroll_top(driver):
    driver.execute_script('window.scrollTo(0, 0)')
    time.sleep(1)
    driver.implicitly_wait(3)


## 페이지 내의 특정 스크롤 끝까지
def scroll_element(driver, cond):
    element = driver.find_element(By.CSS_SELECTOR, cond)
    scroll_location = driver.execute_script('return arguments[0].scrollHeight', element)
    while True:
        driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', element)
        time.sleep(2)
        scroll_height = driver.execute_script('return arguments[0].scrollHeight', element)
        if scroll_location == scroll_height:
            break
        else:
            scroll_location = driver.execute_script('return arguments[0].scrollHeight', element)
    driver.implicitly_wait(3)

In [ ]:
# -------------------- 크롬 준비 --------------------
options = Options()
options.add_experimental_option("detach", True) # 브라우저 꺼짐 방지
options.add_argument('--start-maximized') # 브라우저 최대화 (지정도 가능: --window-size=1920, 1080)

service = Service(excutable_path=ChromeDriverManager().install())

In [23]:
driver = webdriver.Chrome(service=service, options=options)
url = 'https://unipass.customs.go.kr/clip/index.do'
driver.get(url=url)
time.sleep(0.5)

config = {
    'main_kor_dsc': dict(),
    'main_eng_dsc': dict(),
    'all_kor_dsc': dict(),
    'all_eng_dsc': dict(),
}
keywords = customs_data_copy['HS부호_앞4자리'].unique().tolist()

# 세계HS 탭 클릭
btn = driver.find_element(By.XPATH, '//*[@id="topmenu"]/ul/li[3]')
btn.click()
time.sleep(0.5)

# 관세율표 클릭
btn = driver.find_element(By.XPATH, '//*[@id="leftmenu"]/ul/li[1]/ul/li[3]')
btn.click()

for keyword in tqdm(keywords):
    time.sleep(0.5)

    # 검색어 입력
    inp = driver.find_element(By.XPATH, '//*[@id="searchVal"]')
    inp.send_keys(f'{keyword}')
    time.sleep(0.5)

    # 검색 버튼 클릭
    btn = driver.find_element(By.CLASS_NAME, 'btn_inlinesearch')
    btn.click()
    time.sleep(0.5)

    # 핵심 국문 텍스트 수집
    try:
        parse = driver.find_element(By.XPATH, '//*[@id="tblLstBody"]/tr[1]/td[4]').text
        config['main_kor_dsc'][f'{keyword}'] = parse
    except Exception as NoSuchElementException:
        config['main_kor_dsc'][f'{keyword}'] = np.nan

    # 핵심 영문 텍스트 수집
    try:
        parse = driver.find_element(By.XPATH, '//*[@id="tblLstBody"]/tr[1]/td[5]').text
        config['main_eng_dsc'][f'{keyword}'] = parse
    except Exception as NoSuchElementException:
        config['main_eng_dsc'][f'{keyword}'] = np.nan

    scroll(driver) # 전체 페이지 스크롤 끝까지 아래로

    # 국문 텍스트 전체 수집
    scroll_element(driver, '#divLft_tab4') # 특정 페이지 스크롤 끝까지
    parse = driver.find_element(By.XPATH, '//*[@id="divLft_tab4"]').text
    config['all_kor_dsc'][f'{keyword}'] = parse

    # 영문 버튼 클릭
    btn = driver.find_element(By.XPATH, '//*[@id="tab4"]/div/div[1]/span/button[2]')
    btn.click()
    time.sleep(0.5)

    # 영문 텍스트 전체 수집
    scroll_element(driver, '#divLft_tab4') # 특정 페이지 스크롤 끝까지
    parse = driver.find_element(By.XPATH, '//*[@id="divLft_tab4"]').text
    config['all_eng_dsc'][f'{keyword}'] = parse

    scroll_top(driver) # 전체 페이지 스크롤 끝까지 위로

    # 검색어 리셋
    inp = driver.find_element(By.XPATH, '//*[@id="searchVal"]')
    inp.clear()

driver.quit()

# json 형식으로 데이터 저장
with open('./data/customs_crawl_data.json', 'w', encoding='utf-8-sig') as file:
    json.dump(config, file, ensure_ascii=False, indent=4)

100%|██████████| 1352/1352 [3:59:53<00:00, 10.65s/it]  


- 검색되지 않는 부호가 있음 (Ex. 0050, 0007)
    - 이상치인가?

In [48]:
keys = config['all_eng_dsc'].keys()
values0 = config['main_kor_dsc'].values()
values1 = config['main_eng_dsc'].values()
values2 = config['all_kor_dsc'].values()
values3 = config['all_eng_dsc'].values()

data = {
    'HS부호_앞4자리': keys,
    '핵심_한글설명': values0,
    '핵심_영어설명': values1,
    '전체_한글설명': values2,
    '전체_영어설명': values3,
}
crawl_data = pd.DataFrame(data)

In [50]:
crawl_data.head()

,HS부호_앞4자리,핵심_한글설명,핵심_영어설명,전체_한글설명,전체_영어설명
0,0101,살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새,"Live horses, asses, mules and hinnies.",01.01 - 살아 있는 말ㆍ당나귀ㆍ노새ㆍ버새(+)\n- 말\n0101.21 - 번...,"01.01 Live horses, asses, mules and hinnies (+..."
1,0102,살아 있는 소,Live bovine animals.,01.02 - 살아 있는 소(+)\n- 축우(畜牛)\n0102.21 - 번식용\n0...,01.02 Live bovine animals (+).\n- Cattle :\n01...
2,0001,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
3,0103,살아 있는 돼지,Live swine.,01.03 - 살아 있는 돼지(+)\n0103.10 - 번식용\n- 기타\n0103...,01.03 - Live swine (+).\n0103.10 - Pure bred b...
4,0104,살아 있는 면양과 염소,Live sheep and goats.,01.04 - 살아 있는 면양과 염소\n0104.10 - 면양\n0104.20 - ...,01.04 - Live sheep and goats.\n0104.10 - Sheep...


In [49]:
crawl_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호_앞4자리  1352 non-null   object
 1   핵심_한글설명    1229 non-null   object
 2   핵심_영어설명    1229 non-null   object
 3   전체_한글설명    1352 non-null   object
 4   전체_영어설명    1352 non-null   object
dtypes: object(5)
memory usage: 52.9+ KB


In [67]:
crawl_data[crawl_data['핵심_한글설명'].isnull()]

,HS부호_앞4자리,핵심_한글설명,핵심_영어설명,전체_한글설명,전체_영어설명
2,0001,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
14,0002,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
15,0000,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
20,0003,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
21,0030,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
...,...,...,...,...,...
1304,0930,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
1312,0094,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
1318,0095,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.
1319,0950,NaN,NaN,조회결과가 존재하지 않습니다.,조회결과가 존재하지 않습니다.


In [66]:
any(crawl_data[crawl_data['핵심_한글설명'].isnull()].index == crawl_data[crawl_data['핵심_영어설명'].isnull()].index)

True

In [70]:
any(crawl_data[crawl_data['전체_한글설명'] == '조회결과가 존재하지 않습니다.'].index == crawl_data[crawl_data['전체_영어설명'] == '조회결과가 존재하지 않습니다.'].index)

True

In [81]:
temp = []
for idx in range(0, crawl_data.shape[0]):
    if crawl_data.iloc[idx, 4] == ' ': # ''
        temp.append(idx)
print(len(temp))

0


In [82]:
crawl_data.to_csv('./data/customs_crawl_data.csv', index=False, encoding='utf-8-sig')

# 정규식으로 처리

In [6]:
crawl_data = pd.read_csv('./data/customs_crawl_data.csv', dtype=str)

In [7]:
crawl_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1352 entries, 0 to 1351
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HS부호_앞4자리  1352 non-null   object
 1   핵심_한글설명    1229 non-null   object
 2   핵심_영어설명    1229 non-null   object
 3   전체_한글설명    1352 non-null   object
 4   전체_영어설명    1352 non-null   object
dtypes: object(5)
memory usage: 52.9+ KB


In [8]:
keys = crawl_data['HS부호_앞4자리'].tolist()

## 전체_* 컬럼 예시

- 조회결과가 없는 경우, 그대로 두기
- 정보가 담긴 텍스트가 있는 경우, HS부호 관련 정보들 및 소호해설 부분만 제거하기

In [158]:
for idx in range(0, crawl_data.shape[0]):
    if '[소호해설]' in crawl_data.iloc[idx, 3]: # [소호해설], \n◦\n◦ ◦\n, \n*\n* *\n, \n°\n° °\n
        print(idx, end=' ')

0 1 3 5 10 19 22 23 25 32 37 47 55 59 73 75 80 95 100 102 103 104 105 107 108 118 119 124 148 149 152 153 154 155 156 158 170 172 180 199 222 230 242 249 250 257 302 380 381 392 394 397 398 402 405 410 421 423 435 473 482 499 523 528 540 541 544 560 562 573 576 583 590 596 606 608 612 617 628 650 652 655 677 689 721 731 734 740 752 769 774 799 805 821 831 837 845 846 850 851 863 876 879 880 881 885 895 903 904 905 907 908 911 912 913 915 916 918 921 923 925 931 932 933 935 942 946 954 969 972 989 1016 1021 1041 1046 1064 1070 1072 1074 1082 1083 1084 1090 1103 1110 1112 1118 1119 1120 1124 1130 1136 1156 1167 1169 1182 1191 1200 1210 1214 1216 1238 1261 1266 1274 1275 1293 1310 1316 1320 

In [196]:
text = crawl_data.loc[10, '전체_한글설명']
print(text)

02.04 - 면양과 염소의 고기(신선한 것, 냉장하거나 냉동한 것으로 한정한다)(+)
0204.10 - 어린 면양의 도체(屠體)와 이분도체(二分屠體)(신선한 것이나 냉장한 것으로 한정한다)
- 그 밖의 면양의 고기(신선한 것이나 냉장한 것으로 한정한다)
0204.21 - 도체(屠體)와 이분도체(二分屠體)
0204.22 - 그 밖의 것으로서 뼈째로 절단한 것
0204.23 - 뼈 없는 것
0204.30 - 어린 면양의 도체(屠體)와 이분도체(二分屠體)(냉동한 것으로 한정한다)
- 그 밖의 면양의 고기(냉동한 것으로 한정한다)
0204.41 - 도체(屠體)와 이분도체(二分屠體)
0204.42 - 그 밖의 것으로서 뼈째로 절단한 것
0204.43 - 뼈 없는 것
0204.50 - 염소의 고기
이 호에는 면양(숫양ㆍ암양과 어린 면양)ㆍ염소나 어린염소의 신선한 고기, 냉장하거나 냉동한 고기를 분류한다(가축의 것이나 야생의 것인지에 상관없다).
◦
◦ ◦
[소호해설]
소호 제0204.10호와 제0204.30호
소호 제0204.10호와 제0204.30호에서 어린 면양의 고기(meat of lamb)는 생후 12개월 이하의 양에서 얻어진 고기를 말한다. 살코기는 살결이 좋고, 연분홍 붉은색을 띄고 또한 부드러운 겉모습을 가지고 있다. 도체(屠體)의 무게는 26㎏ 이하이다.


In [198]:
pattern_1 = r'\b\d{2}\.\d{2}\b [^\n]+'
pattern_2 = r'\b\d{4}\.\d{2}\b [^\n]+'
pattern_3 = [r'\n◦\n◦ ◦\n', r'\n\*\n\* \*\n', r'\n°\n° °\n', r'\[소호해설\].*']
pattern_4 = r'.*?\n{3,}' # r'- [^\n]+'

text_1 = re.sub(pattern_1, '', text, flags=re.DOTALL).strip()
text_2 = re.sub(pattern_2, '', text_1, flags=re.DOTALL).strip()
temp = text_2
for pat in pattern_3:
    temp = re.sub(pat, '', temp, flags=re.DOTALL).strip()
text_3 = temp
text_4 = re.sub(pattern_4, '', text_3, count=1, flags=re.DOTALL).strip()
print(text_4)

- 그 밖의 면양의 고기(냉동한 것으로 한정한다)




이 호에는 면양(숫양ㆍ암양과 어린 면양)ㆍ염소나 어린염소의 신선한 고기, 냉장하거나 냉동한 고기를 분류한다(가축의 것이나 야생의 것인지에 상관없다).


## 시작!

In [232]:
def preprocessing(df: pd.DataFrame, keywords: list[str]) -> dict[str, dict[str, str]]:
    """
    [Parameters]

    df: 크롤링으로 수집된 데이터가 저장된 csv file
    keywords: 리스트 형식의 HS부호 앞4자리 (str형)
    """
    
    prepro_config = {
        '전처리_한글설명': dict(),
        '전처리_영어설명': dict(),
    }
    
    for idx in tqdm(range(0, df.shape[0])):
        
        key = keywords[idx]

        # 한글
        text = df.iloc[idx, 3]

        if text == '조회결과가 존재하지 않습니다.':
            prepro_config['전처리_한글설명'][key] = text
        else:
            pattern_1 = r'\b\d{2}\.\d{2}\b [^\n]+'
            pattern_2 = r'\b\d{4}\.\d{2}\b [^\n]+'
            pattern_3 = [r'\n◦\n◦ ◦\n', r'\n\*\n\* \*\n', r'\n°\n° °\n', r'\[소호해설\].*']
            pattern_4 = r'\n{2,}' # r'.*?\n{3,}' # r'- [^\n]+'

            text_1 = re.sub(pattern_1, '', text, flags=re.DOTALL).strip()
            text_2 = re.sub(pattern_2, '', text_1, flags=re.DOTALL).strip()
            temp = text_2
            for pat in pattern_3:
                temp = re.sub(pat, '', temp, flags=re.DOTALL).strip()
            text_3 = temp
            text_4 = re.sub(pattern_4, '\n', text_3, flags=re.DOTALL).strip() # repl=''

            prepro_config['전처리_한글설명'][key] = text_4

        # 영어
        text = df.iloc[idx, 4]

        if text == '조회결과가 존재하지 않습니다.':
            prepro_config['전처리_영어설명'][key] = text
        else:
            pattern_1 = r'\b\d{2}\.\d{2}\b [^\n]+'
            pattern_2 = r'\b\d{4}\.\d{2}\b [^\n]+'
            pattern_3 = [r'\n◦\n◦ ◦\n', r'\n\*\n\* \*\n', r'\n°\n° °\n', r'\[소호해설\].*']
            pattern_4 = r'\n{2,}' # r'.*?\n{3,}' # r'- [^\n]+'

            text_1 = re.sub(pattern_1, '', text, flags=re.DOTALL).strip()
            text_2 = re.sub(pattern_2, '', text_1, flags=re.DOTALL).strip()
            temp = text_2
            for pat in pattern_3:
                temp = re.sub(pat, '', temp, flags=re.DOTALL).strip()
            text_3 = temp
            text_4 = re.sub(pattern_4, '\n', text_3, flags=re.DOTALL).strip() # repl=''

            prepro_config['전처리_영어설명'][key] = text_4

    # json 형식으로 데이터 저장
    with open('./data/prepro_crawl_data.json', 'w', encoding='utf-8-sig') as file:
        json.dump(prepro_config, file, ensure_ascii=False, indent=4)

    return prepro_config

In [233]:
df = crawl_data
keywords = crawl_data['HS부호_앞4자리'].tolist()
prepro_config = preprocessing(df, keywords)

100%|██████████| 1352/1352 [00:00<00:00, 2228.47it/s]


In [234]:
keywords[10]

'0204'

In [235]:
print(crawl_data['전체_한글설명'][10])
print('-----'*20)
print(prepro_config['전처리_한글설명']['0204'])

02.04 - 면양과 염소의 고기(신선한 것, 냉장하거나 냉동한 것으로 한정한다)(+)
0204.10 - 어린 면양의 도체(屠體)와 이분도체(二分屠體)(신선한 것이나 냉장한 것으로 한정한다)
- 그 밖의 면양의 고기(신선한 것이나 냉장한 것으로 한정한다)
0204.21 - 도체(屠體)와 이분도체(二分屠體)
0204.22 - 그 밖의 것으로서 뼈째로 절단한 것
0204.23 - 뼈 없는 것
0204.30 - 어린 면양의 도체(屠體)와 이분도체(二分屠體)(냉동한 것으로 한정한다)
- 그 밖의 면양의 고기(냉동한 것으로 한정한다)
0204.41 - 도체(屠體)와 이분도체(二分屠體)
0204.42 - 그 밖의 것으로서 뼈째로 절단한 것
0204.43 - 뼈 없는 것
0204.50 - 염소의 고기
이 호에는 면양(숫양ㆍ암양과 어린 면양)ㆍ염소나 어린염소의 신선한 고기, 냉장하거나 냉동한 고기를 분류한다(가축의 것이나 야생의 것인지에 상관없다).
◦
◦ ◦
[소호해설]
소호 제0204.10호와 제0204.30호
소호 제0204.10호와 제0204.30호에서 어린 면양의 고기(meat of lamb)는 생후 12개월 이하의 양에서 얻어진 고기를 말한다. 살코기는 살결이 좋고, 연분홍 붉은색을 띄고 또한 부드러운 겉모습을 가지고 있다. 도체(屠體)의 무게는 26㎏ 이하이다.
----------------------------------------------------------------------------------------------------
- 그 밖의 면양의 고기(신선한 것이나 냉장한 것으로 한정한다)
- 그 밖의 면양의 고기(냉동한 것으로 한정한다)
이 호에는 면양(숫양ㆍ암양과 어린 면양)ㆍ염소나 어린염소의 신선한 고기, 냉장하거나 냉동한 고기를 분류한다(가축의 것이나 야생의 것인지에 상관없다).


In [239]:
nnn_list = []
for idx in range(0, len(keywords)):
    key = keywords[idx]
    if '\n\n\n' in prepro_config['전처리_한글설명'][key]:
        print(key, end=' ')
        nnn_list.append(key)

In [219]:
for key in nnn_list[:20]:
    print('-----'*20)
    print(prepro_config['전처리_한글설명'][key])

----------------------------------------------------------------------------------------------------
- 말




이 호에는 말(암컷의 말․번식용 말․거세한 말․새끼말․조랑말)․당나귀․노새와 버새를 분류한다(야생이나 가축인지에 상관없다).
노새는 당나귀와 암컷 말과의 잡종이며, 버새는 번식용 말과 당나귀와의 교배종이다.
----------------------------------------------------------------------------------------------------
- 축우(畜牛)


- 버팔로



(1) 축우(畜牛 : cattle)
이 범주에는 보스(Bos)속의 소과 동물들이 포함되는데, 네 가지의 하위 속, 즉, 보스(Bos)․비보스(Bibos)․노비보스(Novibos)와 포에파구스(Poephagus)로 나누어진다. 여기에는 특히 다음과 같은 것들을 포함한다.
(A) 보통의 소(Bos taurus)․혹소나 혹이 있는 소[humped ox(Bos indicus)]와 와츄시 소(Watussi ox)
(B) 비보스(Bibos)속의 아시아 소[예: gaur(Bos gaurus)․gayal(Bos frontalis)와 banteng(Bos sondaicus나 Bos javanicus)]
(C) 포에파구스(Poephagus)속의 동물들[예: 티베트 야크(Bos grunniens)]
(2) 버팔로(buffalo)
이 범주에는 부발루스(Bubalus)속․신세러스(Syncerus)속․비손(Bison)속의 동물들을 포함한다. 여기에는 특히 다음과 같은 것들을 포함한다 :
(A) 부발루스(Bubalus)속의 동물[인도 소나 물소(Bubalus bubalus)․아시아 물소(Asiatic buffalo)나 아니(Bubalus arni)와 셀리비즈 아노아(Celebese anoa)나 피그미 물소(Bubalus depressicornis나 Anoa depressicorn

- `r'\n{2,}' -> '\n'`